In [2]:
import json
from elasticsearch import Elasticsearch

es = Elasticsearch()

In [3]:
def import_to_es(filename):
    with open(filename) as f:
        
        f.readline()
        f.readline()
        list(map(to_es, f))

In [4]:
def to_es(record):
    jsond = json.loads(record[0:-2])
    es.index('grid', 'institutions', jsond, id=jsond['id'])

In [5]:
import_to_es('grid_2015_10_09.json')

JSONDecodeError: Expecting ',' delimiter: line 1 column 1060 (char 1059)

In [25]:
list(actions)

ValueError: I/O operation on closed file.

In [42]:
def eager_import(filename):
    with open(filename) as f:
        records = json.load(f)['institutes']
        for record in records:
            record['num_types'] = len(record.get('types', []))
            es.index('grid', 'institutions', strip_empty(record), id=record['id'])

In [43]:
eager_import('grid_2015_10_09.json')

In [41]:
def strip_empty(document, required=tuple()):
    ''' Removes empty fields from the processed schema
    '''
    new_doc = {}
    for k, v in document.items():
        if k in required:
            new_doc[k] = v
        else:
            new_val = do_strip_empty(v)
            if k == 'otherProperties':
                new_val = [property for property in new_val if property.get('properties')]
            if new_val:
                new_doc[k] = new_val
    return new_doc


def strip_list(l):
    return list(filter(lambda x: x, map(do_strip_empty, l)))


def do_strip_empty(value):
    ''' Filters empty values from container types
    '''
    return {
        dict: strip_empty,
        list: strip_list,
        tuple: strip_list
    }.get(type(value), lambda x: x)(value)
